<a href="https://colab.research.google.com/github/TarkanAl-Kazily/awbw_replay_parser/blob/main/AWBW_Replays_Interactive_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to my AWBW Replay Parser project!

This interactive notebook will introduce you to the replay parsing library, and demonstrate how to use it to learn more about your matches.

This notebook and the remaining source code is all hosted on Github here: [https://github.com/TarkanAl-Kazily/awbw_replay_parser](https://github.com/TarkanAl-Kazily/awbw_replay_parser)

## Prerequisites

Ideally to use this notebook, you have some familiarity with Python and matplotlib if you wish to customize the displayed data you are analyzing. If you wish to contribute and make changes to this notebook or the repo itself, you will need a Github account and make a pull request (following the [contribution guide](https://github.com/TarkanAl-Kazily/awbw_replay_parser))

You will also need an AWBW Replay file that conforms to the newer .zip replay format. You will likely need to have a Google account in order to upload the file for use in the notebook (TODO verify).

ModuleNotFoundError: ignored